In [ ]:
!pip -q install nba_api pandas numpy


In [ ]:
import pandas as pd
import numpy as np
from nba_api.stats.endpoints import leaguegamefinder

SEASON = "2023-24"   # you can change later
TEAM_ABBR = "MIA"    # change to BOS, GSW, NYK, MIA, etc.


In [ ]:
games = leaguegamefinder.LeagueGameFinder(season_nullable=SEASON).get_data_frames()[0]

df = games[games["TEAM_ABBREVIATION"] == TEAM_ABBR].copy()
df["GAME_DATE"] = pd.to_datetime(df["GAME_DATE"])
df = df.sort_values("GAME_DATE").reset_index(drop=True)

df[["GAME_DATE","MATCHUP","WL","PTS","PLUS_MINUS"]].head(10)


,GAME_DATE,MATCHUP,WL,PTS,PLUS_MINUS
0,2023-10-10,MIA vs. CHA,W,113,4.0
1,2023-10-13,MIA @ SAS,L,104,-16.0
2,2023-10-15,MIA vs. MEM,W,132,8.0
3,2023-10-18,MIA vs. BKN,L,104,-3.0
4,2023-10-20,MIA @ HOU,L,104,-6.0
5,2023-10-25,MIA vs. DET,W,103,1.0
6,2023-10-27,MIA @ BOS,L,111,-8.0
7,2023-10-28,MIA @ MIN,L,90,-16.0
8,2023-10-30,MIA @ MIL,L,114,-8.0
9,2023-11-01,MIA vs. BKN,L,105,-4.0


In [ ]:
df["WIN_FLAG"] = (df["WL"] == "W").astype(int)
df["GAME_NUMBER"] = np.arange(1, len(df) + 1)
df["RUNNING_WINS"] = df["WIN_FLAG"].cumsum()
df["RUNNING_WIN_PCT"] = df["RUNNING_WINS"] / df["GAME_NUMBER"]

df["POINT_DIFF"] = df["PLUS_MINUS"]  # point differential proxy

# Opponent + Home/Away from matchup string
df["HOME_AWAY"] = np.where(df["MATCHUP"].str.contains("vs."), "HOME", "AWAY")
df["OPPONENT_ABBR"] = df["MATCHUP"].str.split().str[-1]

df[["GAME_DATE","HOME_AWAY","OPPONENT_ABBR","WIN_FLAG","RUNNING_WIN_PCT","POINT_DIFF"]].head(10)


,GAME_DATE,HOME_AWAY,OPPONENT_ABBR,WIN_FLAG,RUNNING_WIN_PCT,POINT_DIFF
0,2023-10-10,HOME,CHA,1,1.000000,4.0
1,2023-10-13,AWAY,SAS,0,0.500000,-16.0
2,2023-10-15,HOME,MEM,1,0.666667,8.0
3,2023-10-18,HOME,BKN,0,0.500000,-3.0
4,2023-10-20,AWAY,HOU,0,0.400000,-6.0
5,2023-10-25,HOME,DET,1,0.500000,1.0
6,2023-10-27,AWAY,BOS,0,0.428571,-8.0
7,2023-10-28,AWAY,MIN,0,0.375000,-16.0
8,2023-10-30,AWAY,MIL,0,0.333333,-8.0
9,2023-11-01,HOME,BKN,0,0.300000,-4.0


In [ ]:
rng = np.random.default_rng(42)

ARENA_CAPACITY = 19000
df["ARENA_CAPACITY"] = ARENA_CAPACITY

df["DAY_OF_WEEK"] = df["GAME_DATE"].dt.day_name()
df["WEEKEND_FLAG"] = df["DAY_OF_WEEK"].isin(["Friday", "Saturday", "Sunday"]).astype(int)

# demand spikes for "big opponents" (edit as you want)
rival_opps = set(["BOS", "GSW", "DEN", "PHX"])
df["RIVALRY_FLAG"] = df["OPPONENT_ABBR"].isin(rival_opps).astype(int)

# promotions on ~20% of games
df["PROMOTION_FLAG"] = (rng.random(len(df)) < 0.2).astype(int)

# demand score (0.35 to 0.98)
base = 0.62
df["DEMAND_SCORE"] = (
    base
    + 0.10 * df["RUNNING_WIN_PCT"]
    + 0.08 * df["WEEKEND_FLAG"]
    + 0.07 * df["RIVALRY_FLAG"]
    + 0.05 * df["PROMOTION_FLAG"]
).clip(0.35, 0.98)

noise = rng.normal(1.0, 0.04, len(df))
df["TICKETS_SOLD"] = (df["ARENA_CAPACITY"] * df["DEMAND_SCORE"] * noise).round().astype(int)
df["TICKETS_SOLD"] = df["TICKETS_SOLD"].clip(0, df["ARENA_CAPACITY"])

df["AVG_TICKET_PRICE"] = (
    95
    + 35 * df["RIVALRY_FLAG"]
    + 10 * df["WEEKEND_FLAG"]
    + 25 * df["RUNNING_WIN_PCT"]
    + rng.normal(0, 8, len(df))
).round(2)

df["REVENUE"] = (df["TICKETS_SOLD"] * df["AVG_TICKET_PRICE"]).round(2)
df["UTILIZATION_PCT"] = (df["TICKETS_SOLD"] / df["ARENA_CAPACITY"]).round(4)

df[["GAME_DATE","OPPONENT_ABBR","TICKETS_SOLD","AVG_TICKET_PRICE","REVENUE","UTILIZATION_PCT"]].head(10)


,GAME_DATE,OPPONENT_ABBR,TICKETS_SOLD,AVG_TICKET_PRICE,REVENUE,UTILIZATION_PCT
0,2023-10-10,CHA,13448,121.47,1633528.56,0.7078
1,2023-10-13,SAS,14205,107.70,1529878.50,0.7476
2,2023-10-15,MEM,13584,110.72,1504020.48,0.7149
3,2023-10-18,BKN,11993,120.71,1447675.03,0.6312
4,2023-10-20,HOU,14216,128.79,1830878.64,0.7482
5,2023-10-25,DET,12222,106.06,1296265.32,0.6433
6,2023-10-27,BOS,15691,147.65,2316776.15,0.8258
7,2023-10-28,MIN,13505,126.07,1702575.35,0.7108
8,2023-10-30,MIL,13161,94.48,1243451.28,0.6927
9,2023-11-01,BKN,12992,95.34,1238657.28,0.6838


In [ ]:
# dim_date
dim_date = df[["GAME_DATE","DAY_OF_WEEK"]].copy()
dim_date["DATE_KEY"] = dim_date["GAME_DATE"].dt.strftime("%Y%m%d").astype(int)
dim_date["MONTH"] = dim_date["GAME_DATE"].dt.month
dim_date["YEAR"] = dim_date["GAME_DATE"].dt.year
dim_date = dim_date.drop_duplicates().reset_index(drop=True)

# dim_opponent
dim_opponent = df[["OPPONENT_ABBR"]].drop_duplicates().reset_index(drop=True)
dim_opponent["OPPONENT_ID"] = np.arange(1, len(dim_opponent) + 1)

# fact_ticket_sales
fact = df.copy()
fact["DATE_KEY"] = fact["GAME_DATE"].dt.strftime("%Y%m%d").astype(int)
fact = fact.merge(dim_opponent, on="OPPONENT_ABBR", how="left")

fact_ticket_sales = fact[[
    "GAME_ID","DATE_KEY","OPPONENT_ID",
    "HOME_AWAY",
    "TICKETS_SOLD","ARENA_CAPACITY","AVG_TICKET_PRICE","REVENUE","UTILIZATION_PCT",
    "PROMOTION_FLAG","RIVALRY_FLAG",
    "WIN_FLAG","RUNNING_WIN_PCT","POINT_DIFF"
]].copy()

dim_date.head(), dim_opponent.head(), fact_ticket_sales.head()


(   GAME_DATE DAY_OF_WEEK  DATE_KEY  MONTH  YEAR
 0 2023-10-10     Tuesday  20231010     10  2023
 1 2023-10-13      Friday  20231013     10  2023
 2 2023-10-15      Sunday  20231015     10  2023
 3 2023-10-18   Wednesday  20231018     10  2023
 4 2023-10-20      Friday  20231020     10  2023,
   OPPONENT_ABBR  OPPONENT_ID
 0           CHA            1
 1           SAS            2
 2           MEM            3
 3           BKN            4
 4           HOU            5,
       GAME_ID  DATE_KEY  OPPONENT_ID HOME_AWAY  TICKETS_SOLD  ARENA_CAPACITY  \
 0  0012300016  20231010            1      HOME         13448           19000   
 1  0012300033  20231013            2      AWAY         14205           19000   
 2  0012300039  20231015            3      HOME         13584           19000   
 3  0012300056  20231018            4      HOME         11993           19000   
 4  0012300071  20231020            5      AWAY         14216           19000   
 
    AVG_TICKET_PRICE     REVENUE  UT

In [ ]:
# dim_date
dim_date = df[["GAME_DATE","DAY_OF_WEEK"]].copy()
dim_date["DATE_KEY"] = dim_date["GAME_DATE"].dt.strftime("%Y%m%d").astype(int)
dim_date["MONTH"] = dim_date["GAME_DATE"].dt.month
dim_date["YEAR"] = dim_date["GAME_DATE"].dt.year
dim_date = dim_date.drop_duplicates().reset_index(drop=True)

# dim_opponent
dim_opponent = df[["OPPONENT_ABBR"]].drop_duplicates().reset_index(drop=True)
dim_opponent["OPPONENT_ID"] = np.arange(1, len(dim_opponent) + 1)

# fact_ticket_sales
fact = df.copy()
fact["DATE_KEY"] = fact["GAME_DATE"].dt.strftime("%Y%m%d").astype(int)
fact = fact.merge(dim_opponent, on="OPPONENT_ABBR", how="left")

fact_ticket_sales = fact[[
    "GAME_ID","DATE_KEY","OPPONENT_ID",
    "HOME_AWAY",
    "TICKETS_SOLD","ARENA_CAPACITY","AVG_TICKET_PRICE","REVENUE","UTILIZATION_PCT",
    "PROMOTION_FLAG","RIVALRY_FLAG",
    "WIN_FLAG","RUNNING_WIN_PCT","POINT_DIFF"
]].copy()

dim_date.head(), dim_opponent.head(), fact_ticket_sales.head()


(   GAME_DATE DAY_OF_WEEK  DATE_KEY  MONTH  YEAR
 0 2023-10-10     Tuesday  20231010     10  2023
 1 2023-10-13      Friday  20231013     10  2023
 2 2023-10-15      Sunday  20231015     10  2023
 3 2023-10-18   Wednesday  20231018     10  2023
 4 2023-10-20      Friday  20231020     10  2023,
   OPPONENT_ABBR  OPPONENT_ID
 0           CHA            1
 1           SAS            2
 2           MEM            3
 3           BKN            4
 4           HOU            5,
       GAME_ID  DATE_KEY  OPPONENT_ID HOME_AWAY  TICKETS_SOLD  ARENA_CAPACITY  \
 0  0012300016  20231010            1      HOME         13448           19000   
 1  0012300033  20231013            2      AWAY         14205           19000   
 2  0012300039  20231015            3      HOME         13584           19000   
 3  0012300056  20231018            4      HOME         11993           19000   
 4  0012300071  20231020            5      AWAY         14216           19000   
 
    AVG_TICKET_PRICE     REVENUE  UT

In [ ]:
dim_date.to_csv("dim_date.csv", index=False)
dim_opponent.to_csv("dim_opponent.csv", index=False)
fact_ticket_sales.to_csv("fact_ticket_sales.csv", index=False)

print("Saved: dim_date.csv, dim_opponent.csv, fact_ticket_sales.csv")


Saved: dim_date.csv, dim_opponent.csv, fact_ticket_sales.csv


In [ ]:
!pip -q install nba_api pandas numpy scikit-learn


In [ ]:
import pandas as pd
import numpy as np
from nba_api.stats.endpoints import leaguegamefinder

SEASON = "2023-24"
TEAM_ABBR = "MIA"  # keep same as before

games = leaguegamefinder.LeagueGameFinder(season_nullable=SEASON).get_data_frames()[0]
df = games[games["TEAM_ABBREVIATION"] == TEAM_ABBR].copy()

df["GAME_DATE"] = pd.to_datetime(df["GAME_DATE"])
df = df.sort_values("GAME_DATE").reset_index(drop=True)

df.head(3)


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,12023,1610612748,MIA,Miami Heat,0012300016,2023-10-10,MIA vs. CHA,W,241,113,...,0.926,8,43,51,23,10,6,18,25,4.0
1,12023,1610612748,MIA,Miami Heat,0012300033,2023-10-13,MIA @ SAS,L,239,104,...,0.733,19,31,50,23,7,1,23,21,-16.0
2,12023,1610612748,MIA,Miami Heat,0012300039,2023-10-15,MIA vs. MEM,W,240,132,...,0.833,9,38,47,30,3,4,13,21,8.0


In [ ]:
df["WIN_FLAG"] = (df["WL"] == "W").astype(int)
df["GAME_NUMBER"] = np.arange(1, len(df) + 1)
df["RUNNING_WINS"] = df["WIN_FLAG"].cumsum()
df["RUNNING_WIN_PCT"] = df["RUNNING_WINS"] / df["GAME_NUMBER"]

df["POINT_DIFF"] = df["PLUS_MINUS"]

df["HOME_AWAY"] = np.where(df["MATCHUP"].str.contains("vs."), "HOME", "AWAY")
df["OPPONENT_ABBR"] = df["MATCHUP"].str.split().str[-1]
df["DAY_OF_WEEK"] = df["GAME_DATE"].dt.day_name()
df["WEEKEND_FLAG"] = df["DAY_OF_WEEK"].isin(["Friday","Saturday","Sunday"]).astype(int)


In [ ]:
streak = []
current = 0
for w in df["WIN_FLAG"]:
    if w == 1:
        current += 1
    else:
        current = 0
    streak.append(current)

df["WIN_STREAK"] = streak


In [ ]:
big_market = set(["LAL","NYK","GSW","BOS","CHI","MIA"])
df["BIG_MARKET_OPP"] = df["OPPONENT_ABBR"].isin(big_market).astype(int)


In [ ]:
rng = np.random.default_rng(42)

rival_opps = set(["BOS","GSW","DEN","PHX"])
df["RIVALRY_FLAG"] = df["OPPONENT_ABBR"].isin(rival_opps).astype(int)
df["PROMOTION_FLAG"] = (rng.random(len(df)) < 0.2).astype(int)


In [ ]:
ARENA_CAPACITY = 19000
df["ARENA_CAPACITY"] = ARENA_CAPACITY

base = 0.60
df["DEMAND_SCORE"] = (
    base
    + 0.10 * df["RUNNING_WIN_PCT"]
    + 0.05 * (df["WIN_STREAK"] >= 3).astype(int)
    + 0.08 * df["WEEKEND_FLAG"]
    + 0.07 * df["RIVALRY_FLAG"]
    + 0.05 * df["BIG_MARKET_OPP"]
    + 0.05 * df["PROMOTION_FLAG"]
).clip(0.35, 0.98)

noise = rng.normal(1.0, 0.04, len(df))
df["TICKETS_SOLD"] = (df["ARENA_CAPACITY"] * df["DEMAND_SCORE"] * noise).round().astype(int)
df["TICKETS_SOLD"] = df["TICKETS_SOLD"].clip(0, df["ARENA_CAPACITY"])

df["AVG_TICKET_PRICE"] = (
    90
    + 30 * df["RIVALRY_FLAG"]
    + 12 * df["WEEKEND_FLAG"]
    + 18 * df["BIG_MARKET_OPP"]
    + 18 * df["RUNNING_WIN_PCT"]
    + 6  * (df["WIN_STREAK"] >= 3).astype(int)
    + rng.normal(0, 8, len(df))
).round(2)

df["REVENUE"] = (df["TICKETS_SOLD"] * df["AVG_TICKET_PRICE"]).round(2)
df["UTILIZATION_PCT"] = (df["TICKETS_SOLD"] / df["ARENA_CAPACITY"]).round(4)
df["REV_PER_SEAT"] = (df["REVENUE"] / df["ARENA_CAPACITY"]).round(2)

df[["GAME_DATE","OPPONENT_ABBR","TICKETS_SOLD","AVG_TICKET_PRICE","REVENUE","UTILIZATION_PCT","REV_PER_SEAT"]].head(5)


,GAME_DATE,OPPONENT_ABBR,TICKETS_SOLD,AVG_TICKET_PRICE,REVENUE,UTILIZATION_PCT,REV_PER_SEAT
0,2023-10-10,CHA,13075,109.47,1431320.25,0.6882,75.33
1,2023-10-13,SAS,13826,101.20,1399191.20,0.7277,73.64
2,2023-10-15,MEM,13229,103.05,1363248.45,0.6963,71.75
3,2023-10-18,BKN,11635,112.21,1305563.35,0.6124,68.71
4,2023-10-20,HOU,13856,122.99,1704149.44,0.7293,89.69


In [ ]:
from sklearn.linear_model import LinearRegression

features = df[["RUNNING_WIN_PCT","WEEKEND_FLAG","RIVALRY_FLAG","PROMOTION_FLAG","BIG_MARKET_OPP"]]
target = df["REVENUE"]

model = LinearRegression()
model.fit(features, target)

df["PREDICTED_REVENUE"] = model.predict(features).round(2)
df["REVENUE_VARIANCE"] = (df["REVENUE"] - df["PREDICTED_REVENUE"]).round(2)

df[["REVENUE","PREDICTED_REVENUE","REVENUE_VARIANCE"]].head(5)


,REVENUE,PREDICTED_REVENUE,REVENUE_VARIANCE
0,1431320.25,1475834.31,-44514.06
1,1399191.20,1544817.07,-145625.87
2,1363248.45,1612806.28,-249557.83
3,1305563.35,1271866.70,33696.65
4,1704149.44,1546074.52,158074.92


In [ ]:
dim_date = df[["GAME_DATE","DAY_OF_WEEK"]].copy()
dim_date["DATE_KEY"] = dim_date["GAME_DATE"].dt.strftime("%Y%m%d").astype(int)
dim_date["MONTH"] = dim_date["GAME_DATE"].dt.month
dim_date["YEAR"] = dim_date["GAME_DATE"].dt.year
dim_date = dim_date.drop_duplicates().reset_index(drop=True)


In [ ]:
dim_opponent = df[["OPPONENT_ABBR"]].drop_duplicates().reset_index(drop=True)
dim_opponent["OPPONENT_ID"] = np.arange(1, len(dim_opponent) + 1)


In [ ]:
dim_game = df[["GAME_ID","GAME_DATE","HOME_AWAY","MATCHUP","SEASON_ID"]].copy()
dim_game["DATE_KEY"] = dim_game["GAME_DATE"].dt.strftime("%Y%m%d").astype(int)


In [ ]:
df["DATE_KEY"] = df["GAME_DATE"].dt.strftime("%Y%m%d").astype(int)



In [ ]:
fact_team_performance = df.merge(dim_opponent, on="OPPONENT_ABBR", how="left")

fact_team_performance = fact_team_performance[[
    "GAME_ID",
    "DATE_KEY",
    "OPPONENT_ID",
    "WIN_FLAG",
    "RUNNING_WIN_PCT",
    "WIN_STREAK",
    "POINT_DIFF",
    "PTS",
    "PLUS_MINUS"
]].copy()

fact_team_performance.head()


,GAME_ID,DATE_KEY,OPPONENT_ID,WIN_FLAG,RUNNING_WIN_PCT,WIN_STREAK,POINT_DIFF,PTS,PLUS_MINUS
0,0012300016,20231010,1,1,1.000000,1,4.0,113,4.0
1,0012300033,20231013,2,0,0.500000,0,-16.0,104,-16.0
2,0012300039,20231015,3,1,0.666667,1,8.0,132,8.0
3,0012300056,20231018,4,0,0.500000,0,-3.0,104,-3.0
4,0012300071,20231020,5,0,0.400000,0,-6.0,104,-6.0


In [ ]:
tmp = df.merge(dim_opponent, on="OPPONENT_ABBR", how="left")
tmp["DATE_KEY"] = tmp["GAME_DATE"].dt.strftime("%Y%m%d").astype(int)

fact_ticket_sales_v2 = tmp[[
    "GAME_ID","DATE_KEY","OPPONENT_ID",
    "TICKETS_SOLD","ARENA_CAPACITY","AVG_TICKET_PRICE",
    "REVENUE","UTILIZATION_PCT","REV_PER_SEAT",
    "PREDICTED_REVENUE","REVENUE_VARIANCE",
    "PROMOTION_FLAG","RIVALRY_FLAG","BIG_MARKET_OPP"
]].copy()


In [ ]:
dim_date.to_csv("dim_date.csv", index=False)
dim_opponent.to_csv("dim_opponent.csv", index=False)
dim_game.to_csv("dim_game.csv", index=False)
fact_team_performance.to_csv("fact_team_performance.csv", index=False)
fact_ticket_sales_v2.to_csv("fact_ticket_sales_v2.csv", index=False)

print("✅ Exported: dim_date, dim_opponent, dim_game, fact_team_performance, fact_ticket_sales_v2")


✅ Exported: dim_date, dim_opponent, dim_game, fact_team_performance, fact_ticket_sales_v2
